In [9]:
import os
import matplotlib.pyplot
import pandas as pd
import json
from glob import glob
from tqdm import tqdm
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests
import json
from urllib.error import HTTPError
import re
tqdm.pandas()

my_client_id = '4e9cf9cac4224e51b29649eb1ed2b916'
my_secret = '39313a32359e4963aa309b38b46c83fc'

aslan_client_id = '7e5000408e924c139e2f9ec522b3a012'
aslan_secret = '0ff1dc59346749068311a782ce80f7ca'

beka_client_id = 'fe22241e50584325acafcafc80a77adc'
beka_secret = 'e94196def67c46179f130e1800d134a6'

In [10]:
%env SPOTIPY_CLIENT_ID=fe22241e50584325acafcafc80a77adc
%env SPOTIPY_CLIENT_SECRET=e94196def67c46179f130e1800d134a6

env: SPOTIPY_CLIENT_ID=fe22241e50584325acafcafc80a77adc
env: SPOTIPY_CLIENT_SECRET=e94196def67c46179f130e1800d134a6


In [11]:
%env SPOTIPY_CLIENT_ID

'fe22241e50584325acafcafc80a77adc'

In [12]:
%env SPOTIPY_CLIENT_SECRET

'e94196def67c46179f130e1800d134a6'

In [13]:
csv_path = "D:\projects\ProfanityFilter\explicits\songs_info_explicit.csv"
df = pd.read_csv(csv_path)
sp = spotipy.Spotify(
    auth_manager=SpotifyClientCredentials(
    ),
    requests_timeout=20, 
    retries=10
)

In [14]:
def find_last_idx(
    savepath="D:\projects\ProfanityFilter\explicits\explicits_with_two_versions.txt"
):
    if os.path.isfile(savepath) and len(open(savepath).readlines()) > 0:
        idx = open(savepath).readlines()[-1].split(",")[0]
        return int(idx)
    else:
        return 0

def filter_clean_version(expl_info, results):
    expl_artists = set(expl_info.artists_uri.split("|"))
    expl_dur = ''
    clean_dur = ''
    clean_uri = ''
    has_clean_version = False
    
    if "tracks" in results and "items" in results["tracks"]:
        res = results["tracks"]["items"]
        for el in res:
            if el["uri"] == expl_info.uri:
                expl_dur = el["duration_ms"]
                break
            else:
                expl_dur = ''
#         assert expl_dur != '', results # get expl_dur additionally
        
        for el in res:
            clean_artists = set()
            if not el["explicit"]:
                # filter by same artists
                for ar in el["artists"]:
                    clean_artists.add(ar["uri"])
                if clean_artists == expl_artists:
                    # filter by same length
                    clean_dur = el["duration_ms"]
                    has_clean_version = True
                    clean_uri = el["uri"]
                    break
    if expl_info.popularity >= 80:
        has_clean_version = True
    return f"{expl_info.uri},{has_clean_version},{clean_uri},{expl_dur},{clean_dur}\n"

def search_clean_version(row):
    expl_title = row.title
    expl_artist = row.artist
    try:
        results = sp.search(q=f"{expl_artist} {expl_title}", type="track")
    except spotipy.SpotifyException as err:
        if err.code == -1:
            try:
                expl_title = re.sub(r'\-.*$', '', expl_title)
                results = sp.search(q=f"{expl_artist} {expl_title}", type="track")
            except:
                return f'{row.uri},False,,,\n'
        else:
            raise err
    return filter_clean_version(row, results)

In [15]:
savepath="D:\projects\ProfanityFilter\explicits\explicits_with_two_versions.txt"
last_uri = open(savepath).read().splitlines()[-1].split(",")[0]
start = df[df.uri==last_uri].index.item()
start

31122

In [16]:
df = df[start+1:]
pbar = tqdm(df.iterrows(), total=len(df))
for idx, row in pbar:
    info = search_clean_version(row)
    with open(savepath, "a") as wf:
        wf.write(info)

 76%|███████████████████████████████████████████████████████████████████████████████████▍                          | 6775/8931 [42:06<22:07,  1.62it/s]HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'ben folds presents: the sacramento state jazz singers selfless, cold and composed  university a cappella version', 'limit': 10, 'offset': 0, 'type': 'track', 'market': None} returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'ben folds presents: the sacramento state jazz singers selfless, cold and composed  university a cappella version', 'limit': 10, 'offset': 0, 'type': 'track', 'market': None} returned 404 due to Not found.
 87%|████████████████████████████████████████████████████████████████████████████████████████████████              | 7801/8931 [48:26<05:32,  3.40it/s]HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'prince "eye" no, alphabet st., glam slam, anna stesia, dance

In [ ]:
sp.search("brockhampton")